<a href="https://colab.research.google.com/github/dominiksakic/bagOfWords/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  13.1M      0  0:00:06  0:00:06 --:--:-- 16.4M
rm: cannot remove 'aclImdb_v1/train/unsup': No such file or directory


In [3]:
!rm -r aclImdb/train/unsup

In [28]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
for category in ('neg', 'pos'):
  os.makedirs(val_dir / category, exist_ok=True)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:] # from the back
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 6554 files belonging to 2 classes.
Found 18446 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [29]:
from tensorflow.keras import layers

max_length = 600 # Why the max length?
max_tokens = 20000 # Why max tokens?
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

(32, 600)
(32,)
tf.Tensor(
[[  44  319   15 ...    0    0    0]
 [1677    1    3 ...    0    0    0]
 [  11    7    2 ...    0    0    0]
 ...
 [ 362    2   50 ...    0    0    0]
 [4011    1    7 ...    0    0    0]
 [ 733 2436 2767 ...    8 3189    9]], shape=(32, 600), dtype=int64)
tf.Tensor([1 1 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1], shape=(32,), dtype=int32)
